In [1]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

In [10]:
def stock_prices(ticker):
    """
    ticker is the abbreviated symbol for a stock e.g.AAPL
    this function returns the daily price history of the requested stock as a dataframe
    """
    stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    response = requests.get(stock_endpoint + ticker + '?apikey=70407133ea11d7284c70bbca4eee2547').json()
    type(response) == dict
    return pd.DataFrame(response['historical'])

def ny_times_articles(keyword):
    time.sleep(0.1)
    url_list = []
    for i in range(1121):
        response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=news_desk:Business&page="+str(i)+"&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
        docs = response['response']['docs']
        for item in docs:
            url_list.append(item['web_url'])
    return url_list

In [3]:
stock_prices('AAPL')

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2020-12-30,135.580002,135.990005,133.399994,133.720001,133.720001,92882124.0,92882124.0,-1.86000,-1.372,134.37000,"December 30, 20",-0.01372
1,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.869995,120778200.0,120778200.0,-3.18001,-2.304,135.99999,"December 29, 20",-0.02304
2,2020-12-28,133.990005,137.339996,133.509995,136.690002,136.690002,124486200.0,124486200.0,2.70000,2.015,135.84666,"December 28, 20",0.02015
3,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.970001,54930100.0,54930100.0,0.64999,0.495,132.17667,"December 24, 20",0.00495
4,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.960007,88223700.0,88223700.0,-1.20000,-0.908,131.39000,"December 23, 20",-0.00908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2016-01-07,24.670000,25.032499,24.107500,24.112499,22.158121,324377600.0,324377600.0,-0.55750,-2.260,24.41750,"January 07, 16",-0.02260
1255,2016-01-06,25.139999,25.592501,24.967501,25.174999,23.134508,273829600.0,273829600.0,0.03500,0.139,25.24500,"January 06, 16",0.00139
1256,2016-01-05,26.437500,26.462500,25.602501,25.677500,23.596279,223164000.0,223164000.0,-0.76000,-2.875,25.91417,"January 05, 16",-0.02875
1257,2016-01-04,25.652500,26.342501,25.500000,26.337500,24.202784,270597600.0,270597600.0,0.68500,2.670,26.06000,"January 04, 16",0.02670


In [11]:
ny_times_articles('apple')

['https://www.nytimes.com/2017/01/30/technology/personaltech/messages-about-imessages-but-not-from-apple.html',
 'https://www.nytimes.com/2018/06/04/technology/apples-wwdc-live.html',
 'https://www.nytimes.com/2010/04/21/technology/21apple.html',
 'https://dealbook.nytimes.com/2014/05/09/apples-mid-life-crisis/',
 'https://www.nytimes.com/video/technology/100000002023623/for-investors-apple-not-as-shiny.html',
 'https://www.nytimes.com/2013/02/11/technology/samsung-challenges-apples-cool-factor.html',
 'https://bits.blogs.nytimes.com/2014/05/16/apple-and-google-end-patent-fights/',
 'https://www.nytimes.com/2009/06/05/technology/companies/05apple.html',
 'https://www.nytimes.com/video/technology/100000003102972/introducing-apple-watch.html',
 'https://www.nytimes.com/2009/01/22/technology/companies/22apple.html',
 'https://www.nytimes.com/2009/01/22/technology/companies/22apple.html',
 'https://www.nytimes.com/video/technology/1247466754961/demo-of-the-apple-ipad.html',
 'https://bits.

In [14]:
text = []
title = []
abstract = []

headers = {'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
article = requests.get('https://www.nytimes.com/2006/07/01/business/01online.html', headers = headers)
soup = bs4.BeautifulSoup(article.content, 'html.parser')
article_text_p = soup.find_all('p', attrs={'class': 'css-axufdj evys1bk0'})
abstract_text_p = soup.find('p', attrs={'class': 'css-w6ymp8 e1wiw3jv0'})
title_text_h1 = soup.find('h1', attrs={'data-test-id': 'headline'})
temp = []
print(title_text_h1)
title.append(title_text_h1.text)
abstract.append(abstract_text_p.text)
    
for item in article_text_p:
    temp.append(item.text)
space = ' '
article_text = space.join(temp)
text.append(article_text)

<h1 class="css-1fyocly e1h9rw200" data-test-id="headline" id="link-2698b7df">Apple's Got a Secret</h1>


AttributeError: 'NoneType' object has no attribute 'text'